In [7]:
import os
import sys
import random
import pandas
import numpy
import re

In [8]:
HOME_DIR='./'
UNI_ROOT = 'uniamp'
BI_ROOT = 'biamp'
RPS_ROOT = 'rps'
CON_ROOT = 'con'
RPS_PREV_ROOT = 'rps_prev'

ORIGINAL = 'original'
SHADOW = 'shadow'
CANCEL = 'cancel'
LEAKY = 'leaky'
LEAKY_SHADOW = 'leaky_shadow'
PR_HE_LEAK = 7e-7

# Utils

## utils for Utils

In [9]:
# Make sure that the `original`, `shadow` and `cancel` folders are already present and contains main.enum
def create_folders(root, subfolders=['leaky', 'leaky_shadow']):
    # Creates folders
    assert os.path.exists(os.path.join(root, 'original', 'main.pil')), "original circuit doesn't exist."
    assert os.path.exists(os.path.join(root, 'shadow', 'main.pil')), "original circuit doesn't exist."
    assert os.path.exists(os.path.join(root, 'cancel', 'main.pil')), "original circuit doesn't exist."
    for sf in subfolders:
        if not os.path.exists(os.path.join(root, sf)):
            os.mkdir(os.path.join(root, sf))

In [5]:
def read_lines(file):
    with open(file, 'r') as fin:
        lines = fin.readlines()
        lines = [line.strip() for line in lines]
    return lines

def write_lines(lines, file):
    with open(file, 'w') as fout:
        for line in lines:
            fout.write(line)
            fout.write('\n')
            
def eps_ub(d, frac):
    l = [d + d*frac]
    return round(random.choice(l), 9)

def match_replace_rate_const(s, regex='([0-9]\.[0-9]+)', frac=None):
    x = re.search(regex, s)
    if x is not None:
        rep = str(eps_ub(float(x.group()), frac))
        ret = re.sub(regex, rep, s)
        return ret
    else:
        return s

def divide_sections(lines):
    line_numbers = []
    for i, line in enumerate(lines):
        if '#' in line:
            line_numbers.append(i)
    line_numbers.append(len(lines))
    sections = []
    headers = []
    for i in range(len(line_numbers)-1):
        start = line_numbers[i]
        end = line_numbers[i+1]
        sections.append(lines[start+1:end])
        headers.append(lines[start])
    return headers, sections


def merge(ofile, sfile, cfile=None, leaky=False, shadow=False, o_leak_secs=None, s_leak_secs=None):
    olines = read_lines(ofile)
    slines = read_lines(sfile)
    
    o_heds, o_secs = divide_sections(olines)
    s_heds, s_secs = divide_sections(slines)
    
   
    
    assert len(o_secs) == len(s_secs)
    assert len(o_secs) == len(o_leak_secs)
    
    if not leaky:
        headers = o_heds
        sections = o_secs
    else:
        headers = o_heds
        sections = [x[0] + x[1] for x in zip(o_secs, o_leak_secs)]
    
    if leaky and shadow:
        headers = o_heds
        sections = [x[0] + x[1] + x[2] + x[3] for x in zip(o_secs, s_secs, o_leak_secs, s_leak_secs)]
        
        # cancellation reactions
        clines = read_lines(cfile)
        c_heds, c_secs = divide_sections(clines)
        assert len(c_secs) == len(o_secs)
        sections = [x[0] + list(set(x[1]).difference(set(x[0]))) for x in zip(sections, c_secs)]
    
    return headers, sections

def write_out(headers, sections, outfile=None, cancel=False):
    lines = []
    for h, s in zip(headers, sections):
        lines += [h]
        lines += s
    subbed_lines = []
    if cancel:
        for line in lines:
            if re.match(r'shC = hCR fCR mCR sCR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['shC = hCR fCR mCR sCR @initial 0.5 nM']
            elif re.match(r'shA = hAR fAR mAR sAR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['shA = hAR fAR mAR sAR @initial 0.3 nM']
            elif re.match(r'shB = hBR fBR mBR sBR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['shB = hBR fBR mBR sBR @initial 0.2 nM']
            elif re.match(r'shY = hYR fYR mYR sYR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['shY = hYR fYR mYR sYR @initial 0.2 nM']
            else:
                subbed_lines.append(line)
                                
        write_lines(subbed_lines, outfile)
    else:
        write_lines(lines, outfile)

# 1. Unimolecular amplifier

## 1.1 Creating folders

In [5]:
create_folders(UNI_ROOT)

In [6]:
def uni_original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteC = fC( hC( fc( + sC mC fC hC( + sC* ) ) ) ) @initial 0 nM',
        ],
        [
            'macrostate LeakWasteC = [LeakWasteC]',
        ],
        [
            f'reaction [condensed = {PR_HE_LEAK} /nM/s ] ProduceCC + HelperCC -> C + LeakWasteC',
        ]
    ]
    return sections

def uni_shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'shLeakWasteC = fCR*( hCR*( fcR*( hCR*( sCR + ) fCR* mCR* sCR* + ) ) ) @initial 0 nM',
        ],
        [
            'macrostate shLeakWasteC = [shLeakWasteC]',
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] shProduceCC + shHelperCC -> shC + shLeakWasteC',
        ]
    ]
    return sections

In [7]:
ofile = os.path.join(HOME_DIR, UNI_ROOT, ORIGINAL, 'main_enum.pil')
sfile = os.path.join(HOME_DIR, UNI_ROOT, SHADOW, 'main_enum.pil')
cfile = os.path.join(HOME_DIR, UNI_ROOT, CANCEL, 'main_enum.pil')
leaky = os.path.join(HOME_DIR, UNI_ROOT, LEAKY, 'main_enum.pil')
leaky_shadow = os.path.join(HOME_DIR, UNI_ROOT, LEAKY_SHADOW, 'main_enum.pil')

In [8]:
create_folders(os.path.join(HOME_DIR, UNI_ROOT))

In [9]:
headers, sections = merge(ofile, sfile, cfile, leaky=True, o_leak_secs=uni_original_leak_sections(), s_leak_secs=uni_shadow_leak_sections())
write_out(headers, sections, leaky)

headers, sections = merge(ofile, sfile, cfile, leaky=True, shadow=True, o_leak_secs=uni_original_leak_sections(), s_leak_secs=uni_shadow_leak_sections())
write_out(headers, sections, leaky_shadow, cancel=True)

# 2. Bimolecular Amplifier

In [25]:
create_folders(BI_ROOT)

#### Run the original circuit
#### Run the shadow circuit
#### Then run the below file

In [26]:
def bi_original_leak_sections():
    sections = [
        [],
        [],
        [
             'LeakWasteCB = fC( hC( fC( + sC mC fC hC( + sB* ) ) ) ) @initial 0 nM'
        ],
        [
             'macrostate LeakWasteCB = [LeakWasteCB]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceBCC + HelperCC -> C + LeakWasteCB'
        ]
    ]
    return sections

def bi_shadow_leak_sections():
    sections = [
        [],
        [],
        [
             'shLeakWasteCB = fCR*( hCR*( fCR*( hCR*( sBR + ) fCR* mCR* sCR* + ) ) ) @initial 0 nM'
        ],
        [
             'macrostate shLeakWasteCB = [shLeakWasteCB]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] shProduceBCC + shHelperCC -> shC + shLeakWasteCB'
        ]
    ]
    return sections

In [27]:
ofile = os.path.join(HOME_DIR, BI_ROOT, ORIGINAL, 'main_enum.pil')
sfile = os.path.join(HOME_DIR, BI_ROOT, SHADOW, 'main_enum.pil')
cfile = os.path.join(HOME_DIR, BI_ROOT, CANCEL, 'main_enum.pil')
leaky = os.path.join(HOME_DIR, BI_ROOT, LEAKY, 'main_enum.pil')
leaky_shadow = os.path.join(HOME_DIR, BI_ROOT, LEAKY_SHADOW, 'main_enum.pil')

In [28]:
headers, sections = merge(ofile, sfile, cfile, leaky=True, 
                          o_leak_secs=bi_original_leak_sections(), 
                          s_leak_secs=bi_shadow_leak_sections())
write_out(headers, sections, leaky)

headers, sections = merge(ofile, sfile, cfile, leaky=True, shadow=True, 
                          o_leak_secs=bi_original_leak_sections(), 
                          s_leak_secs=bi_shadow_leak_sections())
write_out(headers, sections, leaky_shadow, cancel=True)

# 3. RPS Oscillator

In [29]:
create_folders(RPS_ROOT)

In [30]:
def rps_original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteAC = sA mA fA hA( + sC* ) fA*( hA*( fA*( + ) ) ) @initial 0 nM',
             'LeakWasteBA = sB mB fB hB( + sA* ) fB*( hB*( fB*( + ) ) ) @initial 0 nM',
             'LeakWasteCB = sC mC fC hC( + sB* ) fC*( hC*( fC*( + ) ) ) @initial 0 nM'
        ],
        [
            'macrostate LeakWasteCB = [LeakWasteCB]',
             'macrostate LeakWasteBA = [LeakWasteBA]',
             'macrostate LeakWasteAC = [LeakWasteAC]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceBCC + HelperCC -> C + LeakWasteCB',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceABB + HelperBB -> B + LeakWasteBA',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceCAA + HelperAA -> A + LeakWasteAC'
        ]
    ]
    return sections

def rps_shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'shLeakWasteAC = hAR( fAR mAR sAR + fAR( hAR( fAR( + ) ) ) ) sCR* @initial 0 nM',
             'shLeakWasteBA = hBR( fBR mBR sBR + fBR( hBR( fBR( + ) ) ) ) sAR* @initial 0 nM',
             'shLeakWasteCB = hCR( fCR mCR sCR + fCR( hCR( fCR( + ) ) ) ) sBR* @initial 0 nM'
        ],
        [
            'macrostate shLeakWasteAC = [shLeakWasteAC]',
             'macrostate shLeakWasteBA = [shLeakWasteBA]',
             'macrostate shLeakWasteCB = [shLeakWasteCB]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] shProduceBCC + shHelperCC -> shC + shLeakWasteCB',
             f'reaction [condensed   = {PR_HE_LEAK} /nM/s ] shProduceABB + shHelperBB -> shB + shLeakWasteBA',
             f'reaction [condensed   = {PR_HE_LEAK} /nM/s ] shProduceCAA + shHelperAA -> shA + shLeakWasteAC'
        ]
    ]
    return sections

In [31]:
ofile = os.path.join(HOME_DIR, RPS_ROOT, ORIGINAL, 'main_enum.pil')
sfile = os.path.join(HOME_DIR, RPS_ROOT, SHADOW, 'main_enum.pil')
cfile = os.path.join(HOME_DIR, RPS_ROOT, CANCEL, 'main_enum.pil')
leaky = os.path.join(HOME_DIR, RPS_ROOT, LEAKY, 'main_enum.pil')
leaky_shadow = os.path.join(HOME_DIR, RPS_ROOT, LEAKY_SHADOW, 'main_enum.pil')

In [32]:
headers, sections = merge(ofile, sfile, cfile, leaky=True, 
                          o_leak_secs=rps_original_leak_sections(), 
                          s_leak_secs=rps_shadow_leak_sections())
write_out(headers, sections, leaky)

headers, sections = merge(ofile, sfile, cfile, leaky=True, shadow=True, 
                          o_leak_secs=rps_original_leak_sections(), 
                          s_leak_secs=rps_shadow_leak_sections())
write_out(headers, sections, leaky_shadow, cancel=True)

# 4. Lotka-Volterra Oscillator

In [207]:
def lvo_original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteAC = sA mA fA hA( + sC* ) fA*( hA*( fA*( + ) ) ) @initial 0 nM',
             'LeakWasteBA = sB mB fB hB( + sA* ) fB*( hB*( fB*( + ) ) ) @initial 0 nM',
             'LeakWasteCB = sC mC fC hC( + sB* ) fC*( hC*( fC*( + ) ) ) @initial 0 nM'
        ],
        [
            'macrostate LeakWasteCB = [LeakWasteCB]',
             'macrostate LeakWasteBA = [LeakWasteBA]',
             'macrostate LeakWasteAC = [LeakWasteAC]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceBCjCk + HelperCCk -> Ck + LeakWasteCB',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceABrBs + HelperBBs -> Bs + LeakWasteBA',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceCApAq + HelperAAq -> Aq + LeakWasteAC'
        ]
    ]
    return sections

def lvo_shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'sLeakWasteAC = hAqR( fAR mAR sAR + fAR( hAqR( fAR( + ) ) ) ) sCR* @initial 0 nM',
             'sLeakWasteBA = hBrR( fBR mBR sBR + fBR( hBrR( fBR( + ) ) ) ) sAR* @initial 0 nM',
             'sLeakWasteCB = hCjR( fCR mCR sCR + fCR( hCjR( fCR( + ) ) ) ) sBR* @initial 0 nM'
        ],
        [
            'macrostate sLeakWasteAC = [sLeakWasteAC]',
             'macrostate sLeakWasteBA = [sLeakWasteBA]',
             'macrostate sLeakWasteCB = [sLeakWasteCB]'
        ],
        [
            'reaction [condensed    = 2e-7 /nM/s ] sProduceBCjCj + sHelperCCj -> sCj + sLeakWasteCB',
             'reaction [condensed   = 2e-7 /nM/s ] sProduceABrBr + sHelperBBr -> sBr + sLeakWasteBA',
             'reaction [condensed   = 2e-7 /nM/s ] sProduceCAqAq + sHelperAAq -> sAq + sLeakWasteAC'
        ]
    ]
    return sections

# 5. Consensus

In [33]:
create_folders(CON_ROOT)

In [34]:
def con_original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteYB = fY( hY( fY( + sY mY fY hY( + sB* ) ) ) ) @initial 0 nM',
             'LeakWasteAY = fA( hA( fA( + sA mA fA hA( + sY* ) ) ) ) @initial 0 nM',
             'LeakWasteBY = fB( hB( fB( + sB mB fB hB( + sY* ) ) ) ) @initial 0 nM'
        ],
        [
            'macrostate LeakWasteYB = [LeakWasteYB]',
             'macrostate LeakWasteBY = [LeakWasteBY]',
             'macrostate LeakWasteAY = [LeakWasteAY]'
        ],
        [
            f'reaction [condensed    = 7e-07 /nM/s ] ProduceBYY + HelperYY -> Y + LeakWasteYB',
            f'reaction [condensed    = 7e-07 /nM/s ] ProduceYBB + HelperBB -> B + LeakWasteBY',
            f'reaction [condensed    = 7e-07 /nM/s ] ProduceYAA + HelperAA -> A + LeakWasteAY'
        ]
    ]
    return sections

def con_shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'shLeakWasteYB = fYR( hYR( fYR( + sYR mYR fYR hYR( + sBR* ) ) ) ) @initial 0 nM',
             'shLeakWasteAY = fAR( hAR( fAR( + sAR mAR fAR hAR( + sYR* ) ) ) ) @initial 0 nM',
             'shLeakWasteBY = fBR( hBR( fBR( + sBR mBR fBR hBR( + sYR* ) ) ) ) @initial 0 nM'
        ],
        [
            'macrostate shLeakWasteYB = [shLeakWasteYB]',
             'macrostate shLeakWasteAY = [shLeakWasteAY]',
             'macrostate shLeakWasteBY = [shLeakWasteBY]'
        ],
        [
            f'reaction [condensed    = 7e-07 /nM/s ] shProduceBYY + shHelperYY -> shY + shLeakWasteYB',
            f'reaction [condensed    = 7e-07 /nM/s ] shProduceYBB + shHelperBB -> shB + shLeakWasteBY',
            f'reaction [condensed    = 7e-07 /nM/s ] shProduceYAA + shHelperAA -> shA + shLeakWasteAY'
        ]


    ]
    return sections

In [35]:
ofile = os.path.join(HOME_DIR, CON_ROOT, ORIGINAL, 'main_enum.pil')
sfile = os.path.join(HOME_DIR, CON_ROOT, SHADOW, 'main_enum.pil')
cfile = os.path.join(HOME_DIR, CON_ROOT, CANCEL, 'main_enum.pil')
leaky = os.path.join(HOME_DIR, CON_ROOT, LEAKY, 'main_enum.pil')
leaky_shadow = os.path.join(HOME_DIR, CON_ROOT, LEAKY_SHADOW, 'main_enum.pil')

In [36]:
headers, sections = merge(ofile, sfile, cfile, leaky=True, 
                          o_leak_secs=con_original_leak_sections(), 
                          s_leak_secs=con_shadow_leak_sections())
write_out(headers, sections, leaky)

headers, sections = merge(ofile, sfile, cfile, leaky=True, shadow=True, 
                          o_leak_secs=con_original_leak_sections(), 
                          s_leak_secs=con_shadow_leak_sections())
write_out(headers, sections, leaky_shadow, cancel=True)

# 6. RPS_PREV

In [10]:
create_folders(RPS_PREV_ROOT)

In [12]:
def rps_prev_original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteAC = sA mA fA hA( + sC* ) fA*( hA*( fA*( + ) ) ) @initial 0 nM',
             'LeakWasteBA = sB mB fB hB( + sA* ) fB*( hB*( fB*( + ) ) ) @initial 0 nM',
             'LeakWasteCB = sC mC fC hC( + sB* ) fC*( hC*( fC*( + ) ) ) @initial 0 nM'
        ],
        [
            'macrostate LeakWasteCB = [LeakWasteCB]',
             'macrostate LeakWasteBA = [LeakWasteBA]',
             'macrostate LeakWasteAC = [LeakWasteAC]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceBCjCk + HelperCCk -> Ck + LeakWasteCB',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceABsBr + HelperBBr -> Br + LeakWasteBA',
             f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] ProduceCAqAp + HelperAAp -> Ap + LeakWasteAC'
        ]
    ]
    return sections

def rps_prev_shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'shLeakWasteAC = hAR( fAR mAR sAR + fAR( hAR( fAR( + ) ) ) ) sCR* @initial 0 nM',
             'shLeakWasteBA = hBR( fBR mBR sBR + fBR( hBR( fBR( + ) ) ) ) sAR* @initial 0 nM',
             'shLeakWasteCB = hCR( fCR mCR sCR + fCR( hCR( fCR( + ) ) ) ) sBR* @initial 0 nM'
        ],
        [
            'macrostate shLeakWasteAC = [shLeakWasteAC]',
             'macrostate shLeakWasteBA = [shLeakWasteBA]',
             'macrostate shLeakWasteCB = [shLeakWasteCB]'
        ],
        [
            f'reaction [condensed    = {PR_HE_LEAK} /nM/s ] shProduceBCjCk + shHelperCCk -> shCk + shLeakWasteCB',
             f'reaction [condensed   = {PR_HE_LEAK} /nM/s ] shProduceABsBr + shHelperBBr -> shBr + shLeakWasteBA',
             f'reaction [condensed   = {PR_HE_LEAK} /nM/s ] shProduceCAqAp + shHelperAAp -> shAp + shLeakWasteAC'
        ]

    ]
    return sections

In [13]:
ofile = os.path.join(HOME_DIR, RPS_PREV_ROOT, ORIGINAL, 'main_enum.pil')
sfile = os.path.join(HOME_DIR, RPS_PREV_ROOT, SHADOW, 'main_enum.pil')
cfile = os.path.join(HOME_DIR, RPS_PREV_ROOT, CANCEL, 'main_enum.pil')
leaky = os.path.join(HOME_DIR, RPS_PREV_ROOT, LEAKY, 'main_enum.pil')
leaky_shadow = os.path.join(HOME_DIR, RPS_PREV_ROOT, LEAKY_SHADOW, 'main_enum.pil')

In [14]:
headers, sections = merge(ofile, sfile, cfile, leaky=True, 
                          o_leak_secs=rps_prev_original_leak_sections(), 
                          s_leak_secs=rps_prev_shadow_leak_sections())
write_out(headers, sections, leaky)

headers, sections = merge(ofile, sfile, cfile, leaky=True, shadow=True, 
                          o_leak_secs=rps_prev_original_leak_sections(), 
                          s_leak_secs=rps_prev_shadow_leak_sections())
write_out(headers, sections, leaky_shadow, cancel=True)